# ML Pipeline Demonstration

This notebook demonstrates how to use the ML Pipeline for LGBM classification. It covers the basic functionality of each component and shows how they work together.

In [9]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import the ML pipeline
from ml_pipeline import DataLoader, DataProcessor, ML_Pipeline

# Set up paths
TRAIN_CONFIG_PATH = "data_config_train.json"
TEST_CONFIG_PATH = "data_config_test.json"
MODEL_CONFIG_PATH = "model_config.json"

# Create output directory if it doesn't exist
os.makedirs("trained_model_outputs_path", exist_ok=True)

## 1. Data Loader Demonstration

First, let's demonstrate the DataLoader class which handles loading and validating the data.

In [6]:
metadata_df = pd.read_excel(metadata_path)
metadata_df

,Unnamed: 0,id_col,date_col,target,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13
0,type,category,category,bool,float,float,float,float,float,float,float,float,float,float,category,category,bool,bool
1,use,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,imputation,unknown,unknown,unknown,mean,mean,mean,mean,mean,mean,mean,mean,mean,unknown,mode,unknown,0,unknown
3,category_transformation,None,None,NaN,Standardize,Standardize,Standardize,Standardize,Standardize,Standardize,Standardize,Standardize,Standardize,Standardize,one-hot,None,None,None
4,super_category,1_id,1_id,2_DEPENDENT,3_features,3_features,3_features,3_features,3_features,3_features,3_features,3_features,3_features,3_features,3_features,3_features,3_features,3_features
5,variable_sign,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [8]:
metadata_path = "feature_metadata.xlsx"

metadata_df = pd.read_excel(metadata_path)


# Transform metadata_df to make it more usable
# First row contains column types
feature_types = metadata_df.iloc[0].to_dict()
# Second row contains usage flags
feature_use = metadata_df.iloc[1].to_dict()
# Third row contains imputation methods
feature_imputation = metadata_df.iloc[2].to_dict()
# Fourth row contains category transformations
feature_transformation = metadata_df.iloc[3].to_dict()
# Fifth row contains super categories
feature_super_category = metadata_df.iloc[4].to_dict()

# Create a clean DataFrame for easier access
feature_info = []
for col in metadata_df.columns:
    if col != "Unnamed: 0":
        feature_info.append({
            "feature_name": col,
            "type": feature_types.get(col, ""),
            "use": feature_use.get(col, 0),
            "imputation": feature_imputation.get(col, ""),
            "transformation": feature_transformation.get(col, ""),
            "super_category": feature_super_category.get(col, "")
        })

pd.DataFrame(feature_info)

,feature_name,type,use,imputation,transformation,super_category
0,id_col,category,0,unknown,None,1_id
1,date_col,category,0,unknown,None,1_id
2,target,bool,1,unknown,NaN,2_DEPENDENT
3,feature_0,float,1,mean,Standardize,3_features
4,feature_1,float,1,mean,Standardize,3_features
5,feature_2,float,1,mean,Standardize,3_features
6,feature_3,float,1,mean,Standardize,3_features
7,feature_4,float,1,mean,Standardize,3_features
8,feature_5,float,1,mean,Standardize,3_features
9,feature_6,float,1,mean,Standardize,3_features


In [10]:
# Initialize the data loader with training configuration
data_loader = DataLoader(TRAIN_CONFIG_PATH, MODEL_CONFIG_PATH)

# Get configurations
data_config = data_loader.get_data_config()
model_config = data_loader.get_model_config()

print("Data config:")
for key, value in data_config.items():
    print(f"  {key}: {value}")

print("\nModel config:")
for key, value in model_config.items():
    if key != "hyperparameter_space":  # Skip printing hyperparameter space for brevity
        print(f"  {key}: {value}")

ValueError: Error parsing feature metadata file: Feature metadata file doesn't have the expected structure.

In [4]:
# Get feature metadata
feature_metadata = data_loader.get_feature_metadata()
print("Feature metadata summary:")
print(f"  Total features: {len(feature_metadata)}")
print("\nFeature metadata preview:")
display(feature_metadata.head())

NameError: name 'data_loader' is not defined

In [ ]:
# Get the data with correct data types
data = data_loader.get_data()
print("Data summary:")
print(f"  Shape: {data.shape}")
print(f"  Columns: {list(data.columns)}")
print("\nData preview:")
display(data.head())

print("\nData types:")
display(data.dtypes)

## 2. Data Processor Demonstration

Next, let's demonstrate the DataProcessor class which handles preprocessing the data.

In [ ]:
# Initialize the data processor
data_processor = DataProcessor(
    feature_metadata=feature_metadata,
    model_config=model_config,
    output_dir="trained_model_outputs_path"
)

# Print feature categorization
print("Feature categorization:")
print(f"  Numerical features: {data_processor.numerical_features}")
print(f"  Categorical features: {data_processor.categorical_features}")
print(f"  Binary features: {data_processor.binary_features}")
print(f"  One-hot features: {data_processor.one_hot_features}")
print(f"  Leave as NA features: {data_processor.leave_as_na_features}")

In [ ]:
# Create folds for cross-validation
fold_indices = data_processor.create_folds(data)
print(f"Created {len(fold_indices)} folds for cross-validation")

# Visualize the fold sizes
fold_sizes = [(fold[0].shape[0], fold[1].shape[0]) for fold in fold_indices]
df_fold_sizes = pd.DataFrame(fold_sizes, columns=['train_size', 'val_size'])
df_fold_sizes['fold'] = range(len(fold_indices))

plt.figure(figsize=(10, 6))
plt.bar(df_fold_sizes['fold'], df_fold_sizes['train_size'], label='Train')
plt.bar(df_fold_sizes['fold'], df_fold_sizes['val_size'], bottom=df_fold_sizes['train_size'], label='Validation')
plt.xlabel('Fold')
plt.ylabel('Size')
plt.title('Fold Sizes')
plt.legend()
plt.show()

In [ ]:
# Preprocess the training data
preprocessed_data = data_processor.preprocess_training_data(data, fold_indices)
print("Preprocessed data:")
print(f"  Original data shape: {preprocessed_data['original_data'].shape}")
print(f"  Features shape: {preprocessed_data['X'].shape}")
print(f"  Target shape: {preprocessed_data['y'].shape}")
print(f"  Number of folds: {len(preprocessed_data['folds'])}")

# Print info for the first fold
first_fold = preprocessed_data['folds'][0]
print("\nFirst fold:")
print(f"  Fold index: {first_fold['fold_idx']}")
print(f"  Training indices shape: {first_fold['train_idx'].shape}")
print(f"  Validation indices shape: {first_fold['val_idx'].shape}")
print(f"  X_train shape: {first_fold['X_train'].shape}")
print(f"  X_train_transformed shape: {first_fold['X_train_transformed'].shape}")
print(f"  X_val shape: {first_fold['X_val'].shape}")
print(f"  X_val_transformed shape: {first_fold['X_val_transformed'].shape}")

In [ ]:
# Fit and transform the full dataset
X_transformed, y = data_processor.fit_transform_full_dataset(data)
print("Full dataset transformation:")
print(f"  X_transformed shape: {X_transformed.shape}")
print(f"  y shape: {y.shape}")

# Save the preprocessor
preprocessor_path = data_processor.save_preprocessor()
print(f"Preprocessor saved to: {preprocessor_path}")

## 3. ML Pipeline Demonstration

Finally, let's demonstrate the full ML_Pipeline class which integrates data loading, preprocessing, and model training.

In [ ]:
# Initialize the ML pipeline for training
pipeline = ML_Pipeline(MODEL_CONFIG_PATH, TRAIN_CONFIG_PATH)
print(f"ML Pipeline initialized for {'training' if pipeline.is_training else 'testing'}")

# Run the pipeline for training
pipeline.run()

In [ ]:
# Check the model performances
performances = pipeline.get_model_performances()
print("Model performances:")
for perf in performances:
    print(f"  Fold {perf['fold_idx']}: {perf['performance']}")

# Visualize the performances
plt.figure(figsize=(10, 6))
plt.bar(range(len(performances)), [p['performance'] for p in performances])
plt.axhline(y=np.mean([p['performance'] for p in performances]), color='r', linestyle='-', label='Mean')
plt.xlabel('Fold')
plt.ylabel('Performance')
plt.title('Model Performances Across Folds')
plt.xticks(range(len(performances)), [f"Fold {p['fold_idx']}" for p in performances])
plt.legend()
plt.show()

In [ ]:
# Now initialize the ML pipeline for testing
test_pipeline = ML_Pipeline(MODEL_CONFIG_PATH, TEST_CONFIG_PATH)
print(f"ML Pipeline initialized for {'training' if test_pipeline.is_training else 'testing'}")

# Run the pipeline for testing
test_pipeline.run()

In [ ]:
# Load and display the predictions
predictions_path = test_pipeline.data_config.get("predictions_output_path", "")
if os.path.exists(predictions_path):
    predictions_df = pd.read_csv(predictions_path)
    print("Predictions:")
    print(f"  Shape: {predictions_df.shape}")
    print(f"  Columns: {list(predictions_df.columns)}")
    display(predictions_df.head())
    
    # Visualize the predictions distribution
    plt.figure(figsize=(10, 6))
    plt.hist(predictions_df['prediction'], bins=20)
    plt.xlabel('Prediction')
    plt.ylabel('Count')
    plt.title('Predictions Distribution')
    plt.show()
else:
    print(f"Predictions file not found at: {predictions_path}")

## 4. Optional: Feature Importance Analysis

If desired, we can analyze the feature importances from the trained LGBM model.

In [ ]:
# Load the trained model
import pickle

model_path = os.path.join("trained_model_outputs_path", "target_classification_lgbm.pkl")
if os.path.exists(model_path):
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    
    # Get feature importances
    feature_importance = model.feature_importance()
    
    # Map feature indices to feature names
    feature_indices = list(range(len(feature_importance)))
    
    # Create a dataframe of feature importances
    importance_df = pd.DataFrame({
        'feature_idx': feature_indices,
        'importance': feature_importance
    })
    importance_df = importance_df.sort_values('importance', ascending=False)
    
    # Display feature importances
    print("Feature importances:")
    display(importance_df.head(10))
    
    # Visualize feature importances
    plt.figure(figsize=(12, 8))
    plt.bar(importance_df['feature_idx'].head(20), importance_df['importance'].head(20))
    plt.xlabel('Feature Index')
    plt.ylabel('Importance')
    plt.title('Top 20 Feature Importances')
    plt.xticks(importance_df['feature_idx'].head(20))
    plt.show()
else:
    print(f"Model file not found at: {model_path}")